In [4]:
import os, sys
sys.path.append("../scripts")
import torch
from torch import nn
import re, json, random
import numpy as np
from functools import partial
from tqdm import tqdm
from pprint import pprint
import pandas as pd
pd.set_option('display.float_format', '{:.4f}'.format)
from t5 import *
from helper import *


In [ ]:
#t5_name = 't5-small'
t5_name = 'google/flan-t5-xxl'
t5 = partial(t5_encode_text, name = t5_name, dtype = torch.float32) 
t5("hello world").size()

In [3]:
verbs = json.load(open("../data/coco_verbs/coco_verbs.json", "r"))['transitive']
print(len(verbs))

2073


In [43]:
rel_embs = {
    "top":[],
    "bottom": [],
    "inside": [],
    "outside": [],
    "front": [],
    "behind": [],
    "empty": [],
}
relations = ["is on top of", "is at the bottom of", "is inside", "is outside of", "is in front of", "is behind", "and"]
for k, r in zip(rel_embs.keys(), relations):
    print(k)
    enc = t5(f"A {r} B")[0]
    rel_embs[k].extend([enc[0], enc[-2]])
#relation_embeddings = torch.stack([t5(f"A {r} B")[0, :, :] for r in relations])


top
bottom
inside
outside
front
behind
empty


In [71]:
# compute v-r scores for all v
rel_hats = {"A": [], "B": []}
for v in tqdm(verbs):
    enc = t5(f"A {convert_to_third_person_singular(v)} B")[0]
    rel_hats["A"].append(enc[0]) 
    rel_hats["B"].append(enc[-2]) 
#[A, v1, v2, .., B, </s>]

100%|██████████| 2073/2073 [01:27<00:00, 23.68it/s]


Given r, v

enc("A {r} B") 
enc[0] = A_c + A_l
enc[-2] = B_c + B_l

enc("A {v} B") 
A_c + A_l'
B_c + B_l'

v-r score = sim(A_c + A_l, A_c + A_l') + sim(B_c + B_l, B_c + B_l')


In [72]:
torch.save(rel_hats, "../data/llm_preferences_tmp.pt")
#rel_hats = torch.load("../data/llm_preferences_tmp.pt")

In [73]:
simA = cos_sim(torch.stack([v[0] for v in rel_embs.values()]), torch.stack(rel_hats["A"]))
simB = cos_sim(torch.stack([v[1] for v in rel_embs.values()]), torch.stack(rel_hats["B"]))
print(simA.shape, simB.shape)
sim = (simA + simB)
#sim = sim - sim[-1]
sim = sim.transpose(1,0)
print(sim.shape)

torch.Size([7, 2073]) torch.Size([7, 2073])
torch.Size([2073, 7])


In [62]:
std, mean = torch.std_mean(sim, dim=0)
print(std, mean)
sim = (sim - mean) / std

tensor([0.1293, 0.1111, 0.1384, 0.1136, 0.1255, 0.1245, 0.0000],
       device='cuda:0') tensor([ 0.0129, -0.0022,  0.0654,  0.0486,  0.0140,  0.0719,  0.0000],
       device='cuda:0')


In [74]:
df = pd.DataFrame(columns=["verb"] + list(rel_embs.keys()))
for i, v in enumerate(verbs):
    df.loc[i] = [v] + sim[i].tolist()
df = df.set_index(['verb'])
df.head(5)

,top,bottom,inside,outside,front,behind,empty
verb,,,,,,,
.hit,0.3657,0.3968,0.4900,0.4570,0.3937,0.4591,0.3356
abandon,0.3529,0.4044,0.4301,0.4538,0.3586,0.4474,0.3017
aboarde,0.2880,0.3353,0.4085,0.3707,0.3181,0.3560,0.2955
absorb,0.3782,0.4171,0.4083,0.4669,0.3989,0.4388,0.3450
abuse,0.3180,0.3623,0.4122,0.4295,0.3287,0.4072,0.2802


In [20]:
df.to_csv("../data/llm_preferences/t5_xxl_unnormalized.csv")

In [75]:
for v in ["ride", "support", "inflate", "overturn", "encompass", "wrap", "chase", "pull",]:
    print(df.loc[v])

top       0.4127
bottom    0.4257
inside    0.5402
outside   0.4969
front     0.4209
behind    0.4917
empty     0.3735
Name: ride, dtype: float64
top       0.3450
bottom    0.4069
inside    0.4226
outside   0.4437
front     0.3688
behind    0.4597
empty     0.3013
Name: support, dtype: float64
top       0.2848
bottom    0.3506
inside    0.3942
outside   0.3848
front     0.3089
behind    0.3690
empty     0.2394
Name: inflate, dtype: float64
top       0.4009
bottom    0.4778
inside    0.4425
outside   0.5020
front     0.4177
behind    0.4519
empty     0.3470
Name: overturn, dtype: float64
top       0.3529
bottom    0.4096
inside    0.4164
outside   0.4564
front     0.3737
behind    0.4359
empty     0.2931
Name: encompass, dtype: float64
top       0.3233
bottom    0.3646
inside    0.4003
outside   0.4240
front     0.3338
behind    0.4129
empty     0.2682
Name: wrap, dtype: float64
top       0.4779
bottom    0.4817
inside    0.5000
outside   0.5384
front     0.4828
behind    0.5635
empty  

In [70]:
A = "cat"
B = "mouse"
v = "chase"
rel_embs = {
    "top":[],
    "bottom": [],
    "inside": [],
    "outside": [],
    "front": [],
    "behind": [],
    "empty": [],
}
relations = ["is on top of", "is at the bottom of", "is inside", "is outside of", "is in front of", "is behind", "and"]
for k, r in zip(rel_embs.keys(), relations):
    enc = t5(f"{A} {r} {B}")[0]
    rel_embs[k].extend([enc[0], enc[-2]])
enc = t5(f"{A} {convert_to_third_person_singular(v)} {B}")[0]
vr_scores = {}
for k, x in rel_embs.items():
    vr_scores[k] = cos_sim(x[0], enc[0])[0][0].item() + cos_sim(x[1], enc[1])[0][0].item()
for k, v in vr_scores.items():
    vr_scores[k] -= vr_scores["empty"]
pprint(vr_scores)

{'behind': -0.037854984402656555,
 'bottom': -0.11444016546010971,
 'empty': 0.0,
 'front': -0.04939034581184387,
 'inside': -0.07742635905742645,
 'outside': -0.08330577611923218,
 'top': -0.05273474007844925}


A is on top of B
C is at the bottom of D

A is at the bottom B --> incorrect

In [95]:
v = random.choice(verbs)
print(v)
enc = t5(f"A {convert_to_third_person_singular(v)} B")[0]
print(len(enc))
r_A, r_B = enc[0], enc[-2]
sim = {}
for k, v in rel_embs.items():
    sim[k] = cos_sim(r_A, v[0]) + cos_sim(r_B, v[1])
pprint(sim)

withe
6
{'behind': tensor([[1.1520]], device='cuda:0'),
 'bottom': tensor([[1.1213]], device='cuda:0'),
 'empty': tensor([[1.1276]], device='cuda:0'),
 'front': tensor([[1.1699]], device='cuda:0'),
 'inside': tensor([[1.1858]], device='cuda:0'),
 'outside': tensor([[1.1432]], device='cuda:0'),
 'top': tensor([[1.1110]], device='cuda:0')}


In [7]:
tokenizer = get_tokenizer('google/flan-t5-xxl', torch.float32)
for r in relations:
    texts = [f"A {r} B"]
    encoded = tokenizer.batch_encode_plus(
        texts,
        return_tensors = "pt",
        padding = 'longest',
        max_length = MAX_LENGTH,
        truncation = True
    )
    print(encoded.input_ids)

tensor([[ 71,  19,  30, 420,  13, 272,   1]])
tensor([[  71,   19,   44,    8, 2007,   13,  272,    1]])
tensor([[ 71,  11, 272,   1]])


In [14]:
tokenizer.convert_ids_to_tokens([0, 1, 2, 3, 4, 5])

['<pad>', '</s>', '<unk>', '▁', 'X', '.']

In [10]:
tokenizer.convert_tokens_to_ids(['<s>'])

[2]

In [6]:
tokenizer = get_tokenizer('google/flan-t5-xxl', torch.float32)


In [40]:
RELATION_PHRASES = {
    'top': "is on top of",
    'bottom': "is at the bottom of",
    'front': "is in front of",
    'behind': "is behind",
    'inside': "is inside",
    'outside': "is outside of",
}
texts = list(RELATION_PHRASES.values()) + ["a", "an"]
input_ids = tokenizer.batch_encode_plus(
        texts,
        return_tensors = "pt",
        padding = 'longest',
        max_length = MAX_LENGTH,
        truncation = True
    ).input_ids.tolist()

In [41]:
input_ids

[[19, 30, 420, 13, 1, 0],
 [19, 44, 8, 2007, 13, 1],
 [19, 16, 851, 13, 1, 0],
 [19, 1187, 1, 0, 0, 0],
 [19, 1096, 1, 0, 0, 0],
 [19, 1067, 13, 1, 0, 0],
 [3, 9, 1, 0, 0, 0],
 [46, 1, 0, 0, 0, 0]]

In [44]:
R = []
ARTICALS = []
for y in input_ids[-2:]:
    Y = [str(i) for i in y if i>1]
    ARTICALS.append("${} ".format(" ".join(Y)))
    for x in input_ids[:-2]:
        X = [str(i) for i in x if i>1]
        R.append(" {} ".format(" ".join(X)) + "{} ".format(" ".join(Y)))
print(ARTICALS)
pprint(R)
    

['$3 9 ', '$46 ']
[' 19 30 420 13 3 9 ',
 ' 19 44 8 2007 13 3 9 ',
 ' 19 16 851 13 3 9 ',
 ' 19 1187 3 9 ',
 ' 19 1096 3 9 ',
 ' 19 1067 13 3 9 ',
 ' 19 30 420 13 46 ',
 ' 19 44 8 2007 13 46 ',
 ' 19 16 851 13 46 ',
 ' 19 1187 46 ',
 ' 19 1096 46 ',
 ' 19 1067 13 46 ']


In [56]:
input_ids = tokenizer.batch_encode_plus(
        ["a mountaincat is in front of an sofas"],
        return_tensors = "pt",
        padding = 'longest',
        max_length = MAX_LENGTH,
        truncation = True
    ).input_ids.tolist()
x = "$" + " ".join([str(i) for i in input_ids[0] if i>1])
offsets = [0, 0, 0]
for s in ARTICALS:
    if s in x:
        x = x.replace(s, "")
        offsets[0] = len(s.split())
        break
for s in R:
    if s in x:
        x = x.replace(s, ", ")
        offsets[2] = len(s.split())
        break
offsets[1] = len(x.split(", ")[0].split())
print(x, offsets)
subj_pos, obj_pos = offsets[0], sum(offsets)
print(input_ids[0][subj_pos], input_ids[0][obj_pos])

4180 2138, 5650 7 [2, 2, 5]
4180 5650


In [48]:
tokenizer.convert_ids_to_tokens([71, 1712])

['▁A', '▁cat']